In [1]:
import parser 
import pandas as pd
import itertools
import numpy as np
import os 
import matplotlib.pyplot as plt

In [2]:
def is_consistant(ballot, perm):

    if len(ballot) == 0:
        return True
    c = ballot[0]
    index = 0
    for i in range(len(perm)):
        if (c == perm[i]):
            index = i
            break
    def check(i, j, p):
        '''
        i: index in the ballot
        j: index in the permutation
        p = parent 
        '''
        if j < 0 or j > len(perm):
            return False
        
        if (i == len(ballot) - 1):
            return True
        
        if (i == 0):
            if (j + 1 >= len(perm)):
                if ballot[i + 1] != perm[j - 1]:
                    return False 
            if j - 1 < 0:
                if ballot[i + 1] != perm[j + 1]:
                    return False
            if ballot[i + 1] != perm[j - 1] and ballot[i + 1] != perm[j + 1]:
                return False

        if p != j + 1 and j + 1 < len(perm) and ballot[i + 1] == perm[j + 1]:
            return check(i + 1, j + 1, j)
        elif p != j - 1 and ballot[i + 1] == perm[j - 1]:
            return check(i + 1, j - 1, j)
        else:
            if p != j:
                return check(i, p, p)
            else:
                return False
    return check(0, index, -1)


In [3]:
df = pd.read_csv("MatchedElections.csv")
print(df.query("filename=='Cambridge_11032009_CityCouncil.csv'")["Partisan"])

0    0.0
Name: Partisan, dtype: float64


In [4]:
def dim():
    df = pd.read_csv("MatchedElections.csv")
    directory = "dataverse_files"
    d_p = {}
    d_nonp = {}
    label  = 0
    for filename in os.listdir(directory):
        label += 1
        ballots, candidates = parser.parser(os.path.join(directory, filename))
        x = len(candidates)
        partisan = False
        if x <=6:
            s = "filename==" + filename
            try:
                if df.query(s)["Partisan"] == 0:
                    partisan = False
                else:
                    partisan = True
                if partisan:
                    if x not in d_p:
                        d_p[x] = []
                else:
                    if x not in d_nonp:
                        d_nonp[x] = []
            except:
                continue
            perms = list(itertools.permutations(candidates))
            #most consistant permutation
            mcp = None
            c_mcp = 0
            for p in perms:
                c = 0
                for b in ballots:
                    if (is_consistant(b, p)):
                        c += ballots[b]
                if mcp is None or c > c_mcp:
                    c_mcp = c
                    mcp = p
            if partisan:
                d_p[x].append(c_mcp/sum(ballots.values()))
            else:
                d_nonp[x].append(c_mcp/sum(ballots.values()))
            '''print(filename)
        print("MCP: ", mcp)
        print ("Consistanty: ", c_mcp, "%: ", (100*(c_mcp/sum(ballots.values()))))
        print()'''
    #print(d)
    return d_p, d_nonp



In [5]:
d_p, d_nonp = dim()

In [10]:
print(d_nonp)

dict_items([])


In [6]:
plt.figure(figsize=(10, 6))

for x, y_values in d_p.items():
    for y in y_values:
        plt.scatter(x, y, color='r')

"""for x, y_values in d_nonp.items():
    for y in y_values:
        plt.scatter(x, y, color='b')"""
plt.show()

<Figure size 1000x600 with 0 Axes>

In [7]:
plt.figure(figsize=(10, 6))

for x, y_values in d_p.items():
    for y in y_values:
        plt.scatter(x, y, color='r')

for x, y_values in d_nonp.items():
    for y in y_values:
        plt.scatter(x, y, color='b')
plt.show()

<Figure size 1000x600 with 0 Axes>